In [1]:
from fastai.vision import *
import torch.nn as nn
import pandas as pd
import numpy as np

## 定义数据

In [2]:
root_dir = '/home/he/Data'
df = pd.read_csv('/home/he/Data/CSVs/garbage_classify/info_5_split.csv')
df.head()

,img,label,info
0,DataSets/garbage_classify/img/00_其他垃圾_一次性快餐盒/i...,0,00_其他垃圾_一次性快餐盒
1,DataSets/garbage_classify/img/00_其他垃圾_一次性快餐盒/i...,0,00_其他垃圾_一次性快餐盒
2,DataSets/garbage_classify/img/00_其他垃圾_一次性快餐盒/i...,0,00_其他垃圾_一次性快餐盒
3,DataSets/garbage_classify/img/00_其他垃圾_一次性快餐盒/i...,0,00_其他垃圾_一次性快餐盒
4,DataSets/garbage_classify/img/00_其他垃圾_一次性快餐盒/i...,0,00_其他垃圾_一次性快餐盒


In [3]:
pos = 0
means = int(len(df) / 5)
nums = list(range(0, len(df)))

valid_index = nums[pos*means:(pos+1)*means]
train_index = nums[0:pos*means] + nums[(pos+1)*means:]
len(valid_index), len(train_index)

(2936, 11747)

In [4]:
df = df.iloc[:, [0, 1]]

In [5]:
tfms = get_transforms(do_flip=False, 
                  flip_vert=True, 
                  max_rotate=10, 
                  max_zoom=1, 
                  max_lighting=None, 
                  max_warp=None, 
                  p_affine=0.75, 
                  p_lighting=None)
tfms[0]

[RandTransform(tfm=TfmCrop (crop_pad), kwargs={'row_pct': (0, 1), 'col_pct': (0, 1), 'padding_mode': 'reflection'}, p=1.0, resolved={}, do_run=True, is_random=True, use_on_y=True),
 RandTransform(tfm=TfmAffine (rotate), kwargs={'degrees': (-10, 10)}, p=0.75, resolved={}, do_run=True, is_random=True, use_on_y=True)]

In [6]:
device = torch.device('cuda:1')
data = (ImageList.from_df(df=df, path=root_dir)
        .split_by_idxs(train_idx=train_index, valid_idx=valid_index)
        .label_from_df()
        .transform(size=256, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=48, device=device)
        .normalize(imagenet_stats))

## 定义模型

In [7]:
from torch import nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduction='elementwise_mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduction = reduction

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction is None:
            return F_loss
        else:
            return torch.mean(F_loss)

In [61]:
import pretrainedmodels
def se_resnext50(pretrained=False):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.se_resnext50_32x4d(pretrained=pretrained)
    return model

In [62]:
model = se_resnext50()

In [63]:
model

SENet(
  (layer0): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (layer1): Sequential(
    (0): SEResNeXtBottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se_module): SEModule(
     

In [65]:
model.last_linear.in_features

2048

In [66]:
model.last_linear = nn.Linear(2048, 40)

In [67]:
learn = Learner(data, model, metrics=[accuracy], callback_fns=[ShowGraph],
               )
learn.loss_fn = FocalLoss()

In [68]:
learn.model

SENet(
  (layer0): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (layer1): Sequential(
    (0): SEResNeXtBottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se_module): SEModule(
     

In [56]:
learn.freeze()

In [69]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


RuntimeError: size mismatch, m1: [32 x 8192], m2: [2048 x 40] at /opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/THC/generic/THCTensorMathBlas.cu:273